<ul><li>American Gut Project
</li><li>License: BSD
</li><li>Last Update: April 2014
</li></ul>

The following libraries will need to be installed
</li><li><a href="https://www.python.org/download/releases/2.7/">Python 2.7</a>
</li><li><a href="http://biom-format.org/index.html">BIOM 2.0 or later</a>
</li><li><a href="http://www.numpy.org/">Numpy 1.5.1</a>
</li><li><a href="http://www.scipy.org">SciPy</a>
</li><li><a href="http://pandas.pydata.org">pandas</a>
</li><li><a href="http://statsmodels.sourceforge.net/">statsmodels</a>
</li><li><a href="http://scikit-bio.org/">scikit bio development version</a>
</li><li><a href="http://matplotlib.org/">matplotlib</a>
</li></ul>

Compositional data analysis is a mathematical framework intended to analyze datasets of proportions.  Analyzing datasets of proportions are full of caveats.

For instance, consider a simple example where there are three types of bacteria: A, B, and C.
Initially, all of the bacteria have the same populations.  So the initial sample proportion is $A=1/3$, $B=1/3$, $C=1/3$.

After a period of time, another sample was taken.  This time, the sample proportion was $A=1/2$, $B=1/4$, $C=1/4$.

What happened?

One possibility was that the population of $A$ doubled while the other populations remained the same.
Another possibility was that the population of $B$ and $C$ were halved while the population of $A$ remained constant.
Turns out there are an infinite number of possibilities that could explain the difference in proportions between these two samples.  

Unfortunately, when it comes to microbiome data, all of the datasets are measures of proportions.  The total bacteria count is unknown and one of the most informative measures is the sample estimate of the bacterial proportions.

Compositional data analysis provides an alternative framework that facilities the comparison between samples. This tutorial will illustrate how these analyses can be applied to the American Gut dataset.

In [40]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
from biom import load_table
from skbio.stats.composition import *
from americangut.biplot import make_biplot
import matplotlib.pyplot as plt

Let's load the biom table and the metadata

In [4]:
table = load_table('../data/AG/AG_100nt.biom')
table

22742 x 4827 <class 'biom.table.Table'> with 1891517 nonzero entries (1% dense)

In [5]:
metadata = pd.read_table("../data/AG/AG_100nt.txt", index_col=0, low_memory=False, na_values=['no_data', 'unknown'])

Since we are only analyzing fecal samples, let's filter out the non-fecal samples

In [11]:
metadata = metadata[metadata['BODY_PRODUCT']=='UBERON:feces']

We'll want to compare the healthy subset with the entire population.  The healthy subset will be obtained as follows.

In [12]:
# Defines the filtering functions for alpha diversity
subset_f = {'AGE': lambda x: 19 < x and not np.isnan(x),          
            'DIABETES': lambda x: x == 'I do not have diabetes',
            'IBD': lambda x: x == 'I do not have IBD',
            'ANTIBIOTIC_SELECT': lambda x: x == 'Not in the last year',
            'BMI': lambda x: 18.5 <= x < 30 and not np.isnan(x)}

# Determines which samples meet the requirements of the categories
new_bin = {}
for cat, f in subset_f.iteritems():
    new_bin[cat] = metadata[cat].apply(f)

# Builds up the new binary dataframe
bin_frame = pd.DataFrame(new_bin)

# Adds a column to the current dataframe to look at the subset
bin_series = pd.DataFrame(new_bin).all(1)

# Filters the original mapping file so only samples which satisfy the criteria are included in the new metadata file.
healthy_metadata = metadata.groupby(bin_series).get_group(True)

Since there are > 20,000 OTUs, we'll want to instead analyze the family level.  This will not only speed up calculations, but will also make the biplot a bit more clear.

In [17]:
family_idx=5
collapse_f = lambda id_, md: '; '.join(md['taxonomy'][:family_idx])
family_table = table.collapse(collapse_f, axis='observation')
family_table

634 x 3450 <class 'biom.table.Table'> with 160304 nonzero entries (7% dense)

In [18]:
norm_func = lambda data, id_, md: data / sum(data)
genus_table = genus_table.transform(norm_func, axis='sample')
genus_table

634 x 3450 <class 'biom.table.Table'> with 160304 nonzero entries (7% dense)

Now we'll are going to start making biplots

First, we are going to utilize a singular value decomposition. See page 36 in reference [2] for more details.


In [19]:
mat = genus_table._get_sparse_data().toarray().transpose()
mat = multiplicative_replacement(mat)
mat = centralize(mat)
mat = clr(mat)

In [20]:
L, K, M = np.linalg.svd(mat)

In [22]:
n, _ = L.shape
# Now extract only the first 2 eigenvectors
_k = K[:2]
samp_pca = L[:,:2] * np.sqrt(n-1)
feat_pca = np.multiply(M[:2,:],(_k.reshape(2,1) / np.sqrt(n-1))) 
K[np.isnan(K)]=0
print "%f of the variability is explained"%(sum(_k**2 )/sum(K**2))

0.161391 of the variability is explained


A cartesian to polar coordination conversion is going to be added to aid with the interpretation

In [23]:
def cart2polar(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.degrees(np.arctan2(y, x))
    return (rho, phi)

Pandas dataframes are going to be used, just to keep our data organized

In [31]:
samp_plr = cart2polar(np.ravel(samp_pca[:,0]), np.ravel(samp_pca[:,1]))
feat_plr = cart2polar(np.ravel(feat_pca[0,:]), np.ravel(feat_pca[1,:]))
samp_df = pd.DataFrame({'PCA1':np.ravel(samp_pca[:,0]),
                        'PCA2':np.ravel(samp_pca[:,1]),
                        'radius':samp_plr[0],
                        'degrees':samp_plr[1]},
                        index=genus_table.ids(axis='sample'))

#Include days variable in samp_df
samp_df = pd.merge(samp_df, metadata,
                   left_index=True,right_index=True)

samp_df = samp_df.sort('degrees')

feat_df = pd.DataFrame({'PCA1':np.ravel(feat_pca[0,:]),
                        'PCA2':np.ravel(feat_pca[1,:]),
                        'radius':feat_plr[0],
                        'degrees':feat_plr[1]},
                        index = genus_table.ids(axis='observation'))
feat_df = feat_df.sort('degrees')

Now we're going to construct the biplot

In [45]:
fig = make_biplot(samp_df,feat_df,K,"TYPES_OF_PLANTS",
                  sample_cmap = {-21:'#FFFFFF',
                                -14:'#FFFFFF',
                                 -7:'#FFFFFF',
                                  1:'#FFFFFF',
                                  8:'#999999',
                                 15:'#999999'},
                  otu_cmap={'p__Bacteroidetes':'#219F8D',
                            'p__Firmicutes':'#D04984',
                            'p__Proteobacteria':'#D4D71C',
                            'p__Verrucomicrobia':'#6CAD3F',
                            'p__Derferribateres':'#CF5635',
                            'p__Cyanobacteria':'#FF0000',
                            'p__Tenericutes':'#DFAC35',
                            'p__Chlamydiae':'#7C4A87',
                            'Other':'#1394CA'},
                  alpha=0.5,
                  sample_legend=2,
                  otu_legend=1)

NameError: global name 'plt' is not defined